In [ ]:
from pathlib import Path
from time import monotonic

# test_file = Path.cwd() / "data" / "314_volumes" / "600K_triangles.obj"
test_file = Path.cwd() / "data" / "hongkong" / "level_5_0_0.obj"
assert test_file.exists()

In [ ]:
import trimesh

start = monotonic()
scene: trimesh.Scene = trimesh.load(test_file, force="scene", split_object=True)

for name, mesh in scene.geometry.items():
    print(name)
    verts = mesh.vertices
    tris = mesh.faces
    data = {
        "schema": "objects/triangle-mesh/2.2.0/triangle-mesh",
        "uuid": "123",
        "name": "converted",
        "triangles": {
            "vertices": [{"x": float(x), "y": float(y), "z": float(z)} for x, y, z in verts],
            "indices": [{"n0": int(a), "n1": int(b), "n2": int(c)} for (a, b, c) in tris],
        },
    }

    print(len(data["triangles"]["vertices"]))
    print(len(data["triangles"]["indices"]))

    if isinstance(mesh.visual, trimesh.visual.TextureVisuals):
        material_name = mesh.visual.material.name
        # Trimesh stores texture file in visual.material.image (PIL.Image) if loaded
        # or you can use mesh.visual.material.image.file_path if available
        texture_file = getattr(mesh.visual.material, "image", None)
        print(texture_file)
    elif isinstance(mesh.visual, trimesh.visual.ColorVisuals):
        print(f"Color visual {mesh.visual.face_colors}")
    else:
        print("No material")

print(f"Time taken {monotonic() - start}")
scene.show()

In [ ]:
import open3d as o3d
import numpy as np

start = monotonic()
mesh = o3d.io.read_triangle_mesh(test_file)
mesh.compute_vertex_normals()  # optional

# Vertices and faces
vertices = np.asarray(mesh.vertices)
faces = np.asarray(mesh.triangles)

print("Vertices shape:", vertices.shape)
print("Faces shape:", faces.shape)

data = {
    "schema": "objects/triangle-mesh/2.2.0/triangle-mesh",
    "uuid": "123",
    "name": "converted",
    "triangles": {
        "vertices": [{"x": float(x), "y": float(y), "z": float(z)} for x, y, z in mesh.vertices],
        "indices": [{"n0": int(a), "n1": int(b), "n2": int(c)} for (a, b, c) in mesh.triangles],
    },
}

display(data)

# Material / texture info
materials = []
textures = []

# Open3D stores textures in mesh.textures (list of Image objects)
if len(mesh.textures) > 0:
    for i, tex in enumerate(mesh.textures):
        materials.append(f"Material_{i}")
        textures.append(tex)  # o3d.geometry.Image

print("Materials:", materials)
print("Number of textures:", len(textures))
for i, tex in enumerate(mesh.textures):
    print(f"Texture index {i}: {tex}")

display(np.asarray(mesh.triangle_material_ids)[15])

print(f"Time taken {monotonic() - start}")